In [1]:
import boto3
import tempfile
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()
# spark.stop()
# # spark
# NESSIE_URI = "http://nessie:19120/api/v1"
# MINIO_ACCESS_KEY = "root"
# MINIO_SECRET_KEY = "password"
# conf = (
#     pyspark.SparkConf()
#         .setAppName('app_name')
#         #packages
#         .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.102.5,org.apache.iceberg:iceberg-aws-bundle:1.8.1')
#         #SQL Extensions
#         .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
#         #Configuring Catalog
#         .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
#         .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
#         .set('spark.sql.catalog.nessie.ref', 'main')
#         .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
#         .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
#         .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
#         .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
#         .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
#         #MINIO CREDENTIALS
#         .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
#         .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
# )
# spark = SparkSession.builder.config(conf=conf).getOrCreate()
# spark

25/04/06 17:52:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
%%sql

CREATE DATABASE IF NOT EXISTS poop2

++
||
++
++

In [4]:
s3 = boto3.client('s3', 
        endpoint_url='http://minio:9000',
        aws_access_key_id='admin',
        aws_secret_access_key='password',
        aws_session_token=None,
        config=boto3.session.Config(signature_version='s3v4'),
        verify=False
    )
BUCKET='rawcsv'

In [5]:
# download the file
data = s3.get_object(Bucket=BUCKET, Key='df_open_2023.csv')
# print(data['Body'].read().decode('utf-8').splitlines()[0])
# create a temporary file
with tempfile.NamedTemporaryFile(mode='w',suffix='.csv') as tf:
    # write the downloaded data to the temporary file
    tf.write(data['Body'].read().decode('utf-8'))

    df = spark.read.format("csv").option("header", "true").load(tf.name)
    df.write.saveAsTable("poop2.df_open_2023_lesson3")
    
    # df.createOrReplaceTempView("csv_open_2023")
    # spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()



    # close and delete the temporary file when we're done with it
    tf.close()

In [3]:
%%sql

DESCRIBE EXTENDED poop2.df_open_2023_lesson3

25/04/06 17:52:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/06 17:52:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/04/06 17:52:57 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/04/06 17:52:57 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.18.0.6
25/04/06 17:52:57 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
25/04/06 17:52:57 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


col_name,data_type,comment
competitorId,string,None
competitorName,string,None
firstName,string,None
lastName,string,None
status,string,None
gender,string,None
countryOfOriginCode,string,None
countryOfOriginName,string,None
regionId,string,None
regionName,string,None
